In [1]:
<?php

define('LARAVEL_START', microtime(true));

/*
|--------------------------------------------------------------------------
| Register The Auto Loader
|--------------------------------------------------------------------------
|
| Composer provides a convenient, automatically generated class loader
| for our application. We just need to utilize it! We'll require it
| into the script here so that we do not have to worry about the
| loading of any of our classes manually. It's great to relax.
|
*/

require __DIR__.'/vendor/autoload.php';

$app = require_once __DIR__.'/bootstrap/app.php';

/*
|--------------------------------------------------------------------------
| Run The Artisan Application
|--------------------------------------------------------------------------
|
| When we run the console application, the current CLI command will be
| executed in this console and the response sent back to a terminal
| or another output device for the developers. Here goes nothing!
|
*/

$kernel = $app->make(Illuminate\Contracts\Console\Kernel::class);

$status = $kernel->handle(
    $input = new Symfony\Component\Console\Input\ArgvInput,
    new Symfony\Component\Console\Output\ConsoleOutput
);

<warning>PHP Deprecated:  Using ${var} in strings is deprecated, use {$var} instead in /home/mark/www/housing-prices/vendor/voku/portable-utf8/src/voku/helper/UTF8.php on line 3880</warning>
<warning>PHP Deprecated:  Using ${var} in strings is deprecated, use {$var} instead in /home/mark/www/housing-prices/vendor/voku/portable-utf8/src/voku/helper/UTF8.php on line 4846</warning>
<warning>PHP Deprecated:  Using ${var} in strings is deprecated, use {$var} instead in /home/mark/www/housing-prices/vendor/voku/portable-utf8/src/voku/helper/UTF8.php on line 11473</warning>
<warning>PHP Deprecated:  Using ${var} in strings is deprecated, use {$var} instead in /home/mark/www/housing-prices/vendor/voku/portable-utf8/src/voku/helper/UTF8.php on line 11473</warning>
1


In [2]:
use Illuminate\Console\Command;
use Illuminate\Support\Carbon;
use Illuminate\Support\Facades\DB;
use Rubix\ML\CrossValidation\Metrics\SMAPE;
use Rubix\ML\CrossValidation\Reports\ErrorAnalysis;
use Rubix\ML\Datasets\Labeled;
use Rubix\ML\Datasets\Unlabeled;
use Rubix\ML\GridSearch;
use Rubix\ML\PersistentModel;
use Rubix\ML\Persisters\Filesystem;
use Rubix\ML\Regressors\GradientBoost;
use Rubix\ML\Regressors\RegressionTree;
use Rubix\ML\Backends\Amp;

srand(100);

NULL


In [3]:
$data = DB::table('house_sales')->select(
    'date',
    'bedrooms',
    'bathrooms',
    'sqft_living',
    'sqft_lot',
    'floors',
    'waterfront',
    'view',
    'condition',
    'grade',
    'sqft_above',
    'sqft_basement',
    'yr_built',
    'yr_renovated',
    'zipcode',
    'price'
)->get();

$samples = $data->map(function($sample) {
    return [
        'date' => Carbon::parse($sample->date)->timestamp,
        'bedrooms' => (float) $sample->bedrooms,
        'bathrooms' => (float) $sample->bathrooms,
        'sqft_living' => $sample->sqft_living,
        'sqft_lot' => $sample->sqft_lot,
        'floors' => (float) $sample->floors,
        'waterfront' => $sample->waterfront,
        'view' => (string) $sample->view,
        'condition' => $sample->condition,
        'grade' => $sample->grade,
        'sqft_above' => $sample->sqft_above,
        'sqft_basement' => $sample->sqft_basement,
        'yr_built' => (float) $sample->yr_built,
        'yr_renovated' => (float) $sample->yr_renovated,
        'zipcode' => $sample->zipcode,
    ];
})->toArray();

$labels = $data->pluck('price')->toArray();

$dataset = Labeled::build($samples, $labels);
[$training, $testing] = $dataset->randomize()->split(0.8);

echo "Built datasets";

Built datasets


# Hyper-parameter tuning with GridSearch

In [13]:
$parameters = [
    [PHP_INT_MAX], [1, 5, 10, 15, 20, 25, 40, 80, 150, 250, 500, 1000], [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9]
];

$estimator = new GridSearch(RegressionTree::class, $parameters);
$estimator->setBackend(new Amp(14));

$estimator->train($training);
echo "Training complete";

<warning>PHP Warning:  stream_socket_accept(): Accept failed: Connection timed out in /home/mark/www/housing-prices/vendor/amphp/parallel/lib/Context/Internal/ProcessHub.php on line 78</warning>
<warning>PHP Warning:  stream_socket_accept(): Accept failed: Connection timed out in /home/mark/www/housing-prices/vendor/amphp/parallel/lib/Context/Internal/ProcessHub.php on line 78</warning>
Training complete


In [15]:
$estimator->base()->params();

array (
  'max height' => 9223372036854775807,
  'max leaf size' => 40,
  'max features' => NULL,
  'min purity increase' => 0.0001,
  'max bins' => NULL,
)
